In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import src.preprocessing as prepro
import src.data_handler as data_handler
from src.models import LinealReg

raw_data : data_handler.RawData = data_handler.RawData()

In [2]:
processed_data : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=True, standarize=False, area_units='m2')
processed_data.save_data()
processed_data_nona : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=True, standarize=False, area_units='m2', remove_na_rows=True)
processed_data_nona.save_data(ext='nona')
processed_data_standarized : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=True, standarize=True, area_units='m2')
processed_data_standarized.save_data(ext='standarized')
processed_data_standarized_nona : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=True, standarize=True, area_units='m2', remove_na_rows=True) # Contiene los datos estandarizados, con los tipos corregidos, y unidades unificadas.
processed_data_standarized_nona.save_data(ext='standarized_nona')

4 ) Hago feature engineering a partir de un dataframe con datos procesados no estandarizados

4.1 )

Relleno los datos faltantes con el promedio de los datos

In [3]:
engineered_data : data_handler.ProcessedData = data_handler.ProcessedData(correct_data_types=False, standarize=False, area_units='m2')
engineered_data.fill_missing_values(method='mean')

Nuevas features <br><br>
'high_latitude': Basada en 'lat' y 'lon', reemplazo ambas features con 1 en caso de que la latitud sea alta, y 0 en caso de que la latitud sea baja.<br>
'rooms_per_area': rooms / area<br>
'log_area': log(area)

(Vimos en el punto 1 que siempre que la latitud es alta la longitud es baja).

In [4]:
engineered_data.casas_dev['high_latitude'] = np.where(engineered_data.casas_dev['lat'] > 0, True, False)
engineered_data.casas_dev['rooms_per_area'] = engineered_data.casas_dev['rooms'] / engineered_data.casas_dev['area']
engineered_data.casas_dev['log_area'] = np.log(engineered_data.casas_dev['area'])
engineered_data.casas_dev = engineered_data.casas_dev.drop(columns=['lon', 'lat'])
engineered_data.casas_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   area            900 non-null    float64
 1   area_units      900 non-null    object 
 2   is_house        900 non-null    int64  
 3   has_pool        900 non-null    int64  
 4   age             900 non-null    float64
 5   price           900 non-null    float64
 6   rooms           900 non-null    float64
 7   high_latitude   900 non-null    bool   
 8   rooms_per_area  900 non-null    float64
 9   log_area        900 non-null    float64
dtypes: bool(1), float64(6), int64(2), object(1)
memory usage: 64.3+ KB


In [5]:
train : pd.DataFrame
validation : pd.DataFrame
train, validation = data_handler.get_train_and_validation_sets(engineered_data.casas_dev, train_fraction=0.8, seed=42)
train_standarized : pd.DataFrame = prepro.standarize_numeric_columns(train, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
validation_standarized : pd.DataFrame = prepro.standarize_numeric_columns(validation, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
other_features = ['area', 'price', 'age', 'rooms', 'high_latitude']
train_means : dict[str, float] = {f : train[f].mean() for f in other_features}
train_stds : dict[str, float] = {f : train[f].std() for f in other_features}
validation_means : dict[str, float] = {f : validation[f].mean() for f in other_features}
validation_stds : dict[str, float] = {f : validation[f].std() for f in other_features}
train_standarized.head()

,area,area_units,is_house,has_pool,age,price,rooms,high_latitude,rooms_per_area,log_area
70,-0.454611,m2,0,0,0.191565,0.469669,-0.828209,True,-0.846953,-0.326818
827,0.173135,m2,0,0,-0.709917,0.880550,0.271995,True,0.132921,0.323658
231,1.395587,m2,0,0,-0.033806,-0.734041,1.372199,False,-0.015283,1.295449
588,-0.751964,m2,1,1,-0.033806,0.819786,-0.828209,True,-0.277259,-0.690403
39,1.957254,m2,1,0,1.318418,-0.563323,1.372199,False,-0.610197,1.656530


4.2 )

In [6]:
lin : LinealReg
for caract in ['rooms', 'age', 'high_latitude', 'rooms_per_area', 'log_area']:
    lin : LinealReg = LinealReg(train_standarized[['area', caract]].to_numpy(), train_standarized['price'].to_numpy())
    lin.fit_pseudo_inverse()
    print(f" \"area\", {f"\"{caract}\"":16}", f"|   Error Cuadrático Medio: {lin.error_cuadratico_medio():3f}", f"|   Error Least Square: {lin.error_least_squares_function():3f}")

(720, 3)
(720,)
 "area", "rooms"          |   Error Cuadrático Medio: 0.873407 |   Error Least Square: 628.853028
(720, 3)
(720,)
 "area", "age"            |   Error Cuadrático Medio: 0.847709 |   Error Least Square: 610.350463
(720, 3)
(720,)
 "area", "high_latitude"  |   Error Cuadrático Medio: 0.137116 |   Error Least Square: 98.723250
(720, 3)
(720,)
 "area", "rooms_per_area" |   Error Cuadrático Medio: 0.874282 |   Error Least Square: 629.483010
(720, 3)
(720,)
 "area", "log_area"       |   Error Cuadrático Medio: 0.875313 |   Error Least Square: 630.225587


In [7]:
new_columns : dict[str, float] = {}
other_features = ['rooms', 'age', 'high_latitude', 'rooms_per_area', 'log_area']
for i in range(2, 62):
    other_features.append(f'area^{i}')
    other_features.append(f'age^{i}')
    other_features.append(f'rooms^{i}')
    other_features.append(f'rooms_per_area^{i}')
    other_features.append(f'log_area^{i}')
    new_columns[f'area^{i}'] = engineered_data.casas_dev['area'] ** i
    new_columns[f'age^{i}'] = engineered_data.casas_dev['age'] ** i
    new_columns[f'rooms^{i}'] = engineered_data.casas_dev['rooms'] ** i
    new_columns[f'rooms_per_area^{i}'] = engineered_data.casas_dev['rooms_per_area'] ** i
    new_columns[f'log_area^{i}'] = engineered_data.casas_dev['log_area'] ** i
engineered_data.casas_dev = pd.concat([engineered_data.casas_dev, pd.DataFrame(new_columns)], axis=1)
engineered_data.casas_dev.head()

,area,area_units,is_house,has_pool,age,price,rooms,high_latitude,rooms_per_area,log_area,...,area^60,age^60,rooms^60,rooms_per_area^60,log_area^60,area^61,age^61,rooms^61,rooms_per_area^61,log_area^61
0,59.0,m2,1,0,18.0,546.0,2.0,True,0.033898,4.077537,...,1.782874e+106,2.071812e+75,1.152922e+18,6.466645e-89,4.206079e+36,1.051896e+108,3.729261e+76,2.305843e+18,2.192083e-90,1.715045e+37
1,102.0,m2,0,0,9.0,759.0,3.0,True,0.029412,4.624973,...,3.281031e+120,1.797010e+57,4.239116e+28,1.292007e-92,8.064010e+39,3.346651e+122,1.617309e+58,1.271735e+29,3.800022e-94,3.729583e+40
2,62.0,m2,0,0,7.0,464.0,2.0,True,0.032258,4.127134,...,3.495436e+107,5.080219e+50,1.152922e+18,3.298362e-90,8.687936e+36,2.167170e+109,3.556153e+51,2.305843e+18,1.063988e-91,3.585628e+37
3,127.0,m2,1,1,16.0,251.0,4.0,False,0.031496,4.844187,...,1.691310e+126,1.766847e+72,1.329228e+36,7.859162e-91,1.297940e+41,2.147964e+128,2.826955e+73,5.316912e+36,2.475327e-92,6.287462e+41
4,128.0,m2,0,0,8.0,963.0,4.0,True,0.031250,4.852030,...,2.707685e+126,1.532496e+54,1.329228e+36,4.909093e-91,1.430244e+41,3.465837e+128,1.225996e+55,5.316912e+36,1.534092e-92,6.939586e+41


In [8]:
train : pd.DataFrame
validation : pd.DataFrame
train, validation = data_handler.get_train_and_validation_sets(engineered_data.casas_dev, train_fraction=0.8, seed=42)
train_standarized : pd.DataFrame = prepro.standarize_numeric_columns(train, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
validation_standarized : pd.DataFrame = prepro.standarize_numeric_columns(validation, excluded_columns=set(['lat', 'lon', 'is_house', 'has_pool', 'area_units', 'high_latitude']))
# other_features = ['area', 'price', 'age', 'rooms', 'high_latitude']
train_means : dict[str, float] = {f : train[f].mean() for f in other_features}
train_stds : dict[str, float] = {f : train[f].std() for f in other_features}
validation_means : dict[str, float] = {f : validation[f].mean() for f in other_features}
validation_stds : dict[str, float] = {f : validation[f].std() for f in other_features}
train_standarized.head()

,area,area_units,is_house,has_pool,age,price,rooms,high_latitude,rooms_per_area,log_area,...,area^60,age^60,rooms^60,rooms_per_area^60,log_area^60,area^61,age^61,rooms^61,rooms_per_area^61,log_area^61
70,-0.454611,m2,0,0,0.191565,0.469669,-0.828209,True,-0.846953,-0.326818,...,-0.039968,-0.133052,-0.189531,-0.052743,-0.085053,-0.039858,-0.132617,-0.189530,-0.052743,-0.083293
827,0.173135,m2,0,0,-0.709917,0.880550,0.271995,True,0.132921,0.323658,...,-0.039968,-0.133052,-0.189531,-0.052743,-0.084354,-0.039858,-0.132617,-0.189530,-0.052743,-0.082701
231,1.395587,m2,0,0,-0.033806,-0.734041,1.372199,False,-0.015283,1.295449,...,-0.039968,-0.133052,-0.189522,-0.052743,-0.045816,-0.039858,-0.132617,-0.189524,-0.052743,-0.047847
588,-0.751964,m2,1,1,-0.033806,0.819786,-0.828209,True,-0.277259,-0.690403,...,-0.039968,-0.133052,-0.189531,-0.052743,-0.085089,-0.039858,-0.132617,-0.189530,-0.052743,-0.083322
39,1.957254,m2,1,0,1.318418,-0.563323,1.372199,False,-0.610197,1.656530,...,-0.039968,-0.133039,-0.189522,-0.052743,0.075765,-0.039858,-0.132607,-0.189524,-0.052743,0.065431


In [ ]:
# print(train_standarized['price'].to_numpy().shape)
# print(validation_standarized['price'].to_numpy().shape)
lin : LinealReg = LinealReg(train_standarized[other_features].to_numpy(), train_standarized['price'].to_numpy())
# lin.fit_pseudo_inverse()
lin.fit_gradient_descent(step_size=0.000005, tolerance=-1, max_number_of_steps=1000)
print(lin.error_cuadratico_medio())
# print("1: ", validation_standarized['price'].to_numpy())
# print(lin.error_cuadratico_medio(validation_set_x=validation_standarized[other_features].to_numpy(), validation_set_y=validation_standarized['price'].to_numpy()))
# lin.fit_pseudo_inverse()

(720, 306)
(720,)
0.15744674720526752
(720, 306)
(720,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 306 is different from 305)